In [5]:
import os

In [6]:
%pwd

'c:\\Users\\Suel.Abbasi\\Desktop\\DSA\\projects\\end to end ml project\\end-to-end-ML-Project-with-mlFlow\\research'

In [7]:
os.chdir('../')

In [8]:
%pwd

'c:\\Users\\Suel.Abbasi\\Desktop\\DSA\\projects\\end to end ml project\\end-to-end-ML-Project-with-mlFlow'

In [36]:
# entity

from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path 
    unzip_dir: Path 

In [37]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [46]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config =self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url= config.source_url,
            local_data_file= config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [47]:
import os 
import urllib.request as request 
import zipfile
from mlProject import logger 
from mlProject.utils.common import get_size

In [50]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url= self.config.source_url,
                filename= self.config.local_data_file
            )
            logger.info(f'{filename} download! With the following info: \n{headers}')
        else:
            logger.info(f'file already exists of size: {get_size(Path(self.config.local_data_file))}')
            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [51]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e: 
    raise e

[2025-02-28 22:23:34,316: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-28 22:23:34,316: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-28 22:23:34,322: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-28 22:23:34,323: INFO: common: created directory at: artifacts]
[2025-02-28 22:23:34,323: INFO: common: created directory at: artifacts/data_ingestion]
[2025-02-28 22:23:34,332: INFO: 1748763302: file already exists of size: ~ 23 KB]
